In [1]:
from LLMstoDataBase import LLMsToDataBase, SQLAgent
from open_ai_utils import enviar_promt_chat_completions_mode, get_embedding, simular_respuesta_generativa
from calcular_tokens import num_tokens_from_string, num_tokens_from_messages

import warnings

# Ignorar todos los warnings
warnings.filterwarnings('ignore')

import pandas as pd
import openai
import numpy as np
import os

In [2]:
from dotenv import load_dotenv

# Cargar variables de entorno desde .env
load_dotenv()

# Acceder a la API key
api_key = os.getenv("API_KEY")
openai.api_key = api_key

# probando el pipeline paso a paso
## Carga de metadatos de la BBDD

Si el archivo.pickle existe en el directorio con los metadatos nos dice que ya existe, de lo contrario, los extrae de la bbdd. LA BASE DE DATOS DEBE ESTAR LEVANTADA 

In [3]:
rental = {
    'bbdd_name':'dvdrental', 
    'user'     :'postgres', 
    'password' :'123', 
    'host'     :'localhost', 
    'port'     :'5432'
}

rna_central = {
    'bbdd_name': 'pfmegrnargs', 
    'user'     : 'reader', 
    'password' : 'NWDMCE5xdipIjRrp', 
    'host'     : 'hh-pgsql-public.ebi.ac.uk', 
    'port'     : '5433'
}


llm_to_pagila = LLMsToDataBase(**rental)
llm_to_pagila.store_metadata_df_to_pickle()
#llm_to_pagila.close_connection()


llm_to_rna = LLMsToDataBase(**rna_central)
llm_to_rna.store_metadata_df_to_pickle()
#llm_to_rna.close_connection()


Archivo guardado: ../data/nlp_to_sql/dvdrental_simple_metadata.pickle
Archivo guardado: ../data/nlp_to_sql/dvdrental_full_metadata.pickle

Proceso de almacenamiento de metadatos para dvdrental completado.

El archivo ya existe en: ../data/nlp_to_sql/pfmegrnargs_simple_metadata.pickle
El archivo ya existe en: ../data/nlp_to_sql/pfmegrnargs_full_metadata.pickle

Proceso de almacenamiento de metadatos para pfmegrnargs completado.



In [5]:
data_path = '../data/nlp_to_sql/'

simple_md_df = pd.read_pickle(data_path + 'dvdrental_simple_metadata.pickle')   # pfmegrnargs   # pagila
full_md_df = pd.read_pickle(data_path + 'dvdrental_full_metadata.pickle')

display(simple_md_df.head())
display(full_md_df.head())

,metadata_str,md_str_tokens,embedding
actor,"{""actor"": [""actor_id"", ""first_name"", ""last_nam...",19,"[-0.0329330638051033, -0.005584632977843285, 0..."
store,"{""store"": [""store_id"", ""manager_staff_id"", ""ad...",20,"[-0.029232481494545937, 0.008416695520281792, ..."
address,"{""address"": [""address_id"", ""address"", ""address...",32,"[-0.027407445013523102, 0.02479003369808197, 0..."
city,"{""city"": [""city_id"", ""city"", ""country_id"", ""la...",18,"[-0.011713930405676365, 0.020075011998414993, ..."
country,"{""country"": [""country_id"", ""country"", ""last_up...",14,"[-0.0247060414403677, 0.011358940042555332, 0...."


,metadata_str,md_str_tokens,embedding
actor,"table_name:actor\n{""col_name"": ""actor_id"", ""ty...",138,"[-0.003070953767746687, 0.0058361561968922615,..."
store,"table_name:store\n{""col_name"": ""store_id"", ""ty...",139,"[-0.006375625263899565, 0.021333562210202217, ..."
address,"table_name:address\n{""col_name"": ""address_id"",...",271,"[-0.0031046976801007986, 0.025939086452126503,..."
city,"table_name:city\n{""col_name"": ""city_id"", ""type...",137,"[0.009170434437692165, 0.023855293169617653, 0..."
country,"table_name:country\n{""col_name"": ""country_id"",...",103,"[0.00015362424892373383, 0.014917444437742233,..."


In [6]:
print('El numero de tablas que tiene la actual base de datos es: ')
print(len(simple_md_df), len(full_md_df))

El numero de tablas que tiene la actual base de datos es: 
15 15


## Buscando metadatos más similares a una consulta

In [7]:
# consulta facil - medio
consulta_pagila = '''
dame los nombres de los actores y actrices que hayan participado en 
peliculas con una duracion superior 180 minutos. incluye tambien la 
duración exacta de la pelicula
'''
# consulta facil
consulta_pagila_01 = '''cuantas peliculas tienen la duración maxima y cuanta es esa duración'''

# consulta dificil media
consulta_pagila_02 = '''Los nombres de los actores que hayan participado en más de 3 peliculas, \
                        cuyo nombre empieze por una vocal, y que las peliculas duren más de 180 minutos'''

#consulta dificil
consulta_pagila_03 = '''¿Que precio tienen las peliculas en las que ha trabajado el actor o actores con más películas, \
                        devuelve el nombre de las películas y el actor o actores que hayan trabajado en ellas?'''

# consulta_rna_central_1 = "Muestra todas las secuencias de ARN que tienen una longitud mayor a 200 nucleótidos."
# consulta_rna_central_2 =  "Encuentra las entradas de ARN que han sido identificadas en organismos específicos como 'Homo sapiens'."
# consulta_rna_central_3 = "Lista los estudios que han citado secuencias de ARN particulares más de 50 veces."
# consulta_rna_central_4 =  "Obtén los nombres de los investigadores que han contribuido con más de 10 secuencias de ARN a la base de datos."
# consulta_rna_central_5 = "Muestra las secuencias de ARN que se han descubierto en el último año."

token_limit = 1000
n_tablas = None

most_similar_md_simple = LLMsToDataBase.search_most_similar_metadata(
    prompt= consulta_pagila, 
    metadata=simple_md_df, 
    n_resultados=n_tablas, 
    lim_tokens=token_limit)

most_similar_md_full = LLMsToDataBase.search_most_similar_metadata(
    prompt= consulta_pagila, 
    metadata=full_md_df, 
    n_resultados=n_tablas, 
    lim_tokens=token_limit )

display(most_similar_md_simple)
display(most_similar_md_full)

,metadata_str,md_str_tokens,similarity,embedding,token_cumsum
film_actor,"{""film_actor"": [""actor_id"", ""film_id"", ""last_u...",16,0.732587,"[-0.033507075160741806, 0.003144766902551055, ...",16
film,"{""film"": [""film_id"", ""title"", ""description"", ""...",53,0.732268,"[-0.010311612859368324, 0.012414505705237389, ...",69
actor,"{""actor"": [""actor_id"", ""first_name"", ""last_nam...",19,0.725583,"[-0.0329330638051033, -0.005584632977843285, 0...",88
film_category,"{""film_category"": [""film_id"", ""category_id"", ""...",16,0.699847,"[-0.02557847648859024, 0.014861471951007843, -...",104
inventory,"{""inventory"": [""inventory_id"", ""film_id"", ""sto...",19,0.691929,"[-0.017722778022289276, 0.0021708004642277956,...",123
language,"{""language"": [""language_id"", ""name"", ""last_upd...",14,0.686570,"[-0.02983446605503559, 0.018869545310735703, 0...",137
rental,"{""rental"": [""rental_id"", ""rental_date"", ""inven...",34,0.683688,"[-0.02543514594435692, 0.002907260088250041, 0...",171
staff,"{""staff"": [""staff_id"", ""first_name"", ""last_nam...",42,0.676197,"[-0.01896057464182377, 0.008822894655168056, -...",213
category,"{""category"": [""category_id"", ""name"", ""last_upd...",14,0.674468,"[-0.030227964743971825, 0.017394820228219032, ...",227
address,"{""address"": [""address_id"", ""address"", ""address...",32,0.674441,"[-0.027407445013523102, 0.02479003369808197, 0...",259


,metadata_str,md_str_tokens,similarity,embedding,token_cumsum
actor,"table_name:actor\n{""col_name"": ""actor_id"", ""ty...",138,0.742149,"[-0.003070953767746687, 0.0058361561968922615,...",138
film_actor,"table_name:film_actor\n{""col_name"": ""actor_id""...",107,0.740670,"[-0.005460824351757765, 0.010005425661802292, ...",245
film,"table_name:film\n{""col_name"": ""film_id"", ""type...",443,0.723553,"[-0.0006646884721703827, 0.02408052608370781, ...",688
film_category,"table_name:film_category\n{""col_name"": ""film_i...",107,0.718267,"[0.0017928910674527287, 0.02142120897769928, -...",795
inventory,"table_name:inventory\n{""col_name"": ""inventory_...",138,0.709335,"[0.00014501469559036195, 0.015438942238688469,...",933


In [ ]:
print(len(most_similar_md_simple))
print(len(most_similar_md_full))

In [ ]:
simple_md_str = LLMsToDataBase.get_string_from_metadata_df(most_similar_md_simple)
full_md_str = LLMsToDataBase.get_string_from_metadata_df(most_similar_md_full)

print('SIMPLE METADATA')
print(simple_md_str)
print('\n')
print('FULL METADATA')
print(full_md_str)

# Probando el agente

## Lo probamos en dos consultas

In [ ]:
consulta ='''
¿Que precio tienen las peliculas en las que ha trabajado el actor o actores con más películas, \
devuelve el nombre de las películas y el actor o actores que hayan trabajado en ellas?
'''

agent = SQLAgent(**rental)
# print('conexion')
# print(agent.bbdd_name)
# print(agent.historico.contador_interacciones)

tabla_resultado, codigo_sql = agent.nlp_to_sql(consulta_nlp=consulta, metadata_token_limit= 1000)

## Probando el metodo 'continuar_conversacion'

### Con una nueva consulta

In [ ]:
continuar = '''dame los nombres de los actores y actrices que hayan participado en 
peliculas con una duracion superior 180 minutos. incluye tambien la 
duración exacta de la pelicula'''

cont, nueva_consulta = agent.continuar_conversando(
    usuario=continuar, 
    codigo_sql_ejecutado=codigo_sql, 
    tabla_consulta_anterior=tabla_resultado
)

print(cont, nueva_consulta)

### Con una duda sobre la consulta anterior

In [ ]:
duda = '''Cuales de los actores mencionados, tienen un nombre que empieza por una vocal'''

cont, nueva_consulta = agent.continuar_conversando(
    usuario= duda, 
    codigo_sql_ejecutado= '', 
    tabla_consulta_anterior= tabla_resultado
)

print(cont, nueva_consulta)

### Texto random

In [ ]:
duda = '''como se hacen los macarrones?'''

cont, nueva_consulta = agent.continuar_conversando(
    usuario=duda, 
    codigo_sql_ejecutado=codigo_sql, 
    tabla_consulta_anterior=tabla_resultado
)

print(cont, nueva_consulta)

## añadimos una nueva consulta

In [ ]:
consulta ='''dame los nombres de los actores y actrices que hayan participado en 
peliculas con una duracion superior 180 minutos. incluye tambien la 
duración exacta de la pelicula'''

#agent = SQLAgent(**pagila)
# print('conexion')
# print(agent.bbdd_name)
# print(agent.historico.contador_interacciones)

tabla_resultado, codigo_sql = agent.nlp_to_sql(consulta_nlp=consulta, metadata_token_limit= 1000)
agent.close_connection()
# print('connexion cerrada')

# # simular_respuesta_generativa(tabla_resultado)

## generar informe

In [ ]:
informe = agent.informe_resultado(
    consulta_usuario=consulta, 
    codigo_sql= codigo_sql, 
    tabla_texto= tabla_resultado, 
    max_tokens_respuesta= 1000
)

simular_respuesta_generativa(informe)

In [ ]:
print(num_tokens_from_string(informe))

## Probando la duda sobre consutla anterior

In [ ]:
duda = '''de los actores mencionados, dame los nombres de cuyos apellidos empiezen por una vocal'''

respuesta_duda = agent.pregunta_sobre_consulta_anterior(
    usuario= duda, 
    tabla_consulta_anterior=None, # le podemos pasar la tabla o no, lo más segura es pasarla pero tambien es más costoso 
    max_tokens_historico= 1500, 
    max_tokens_respuesta=300
)


print(respuesta_duda)

## Probando el historico

### historico completo

In [ ]:
print(agent.historico_completo.historico)
num_tokens_from_string(agent.historico_completo.historico)

### Historico simple

Esto solo guarda las consultas del usuario y el informe generado por el LLM. Hay que tener en cuenta que en el flujo de dialogo real despues de una nueva consulta siempre se generará un informe por lo que siempre ira una respuesta después de una consulta del usuario

In [ ]:
print(agent.historico.historico)

## Historico estructurado

Hay que tener en cuenta que se guarda la información estrucutrada cada vez que responde el LLM en este caso no ha sido así, por eso vemos la incoherencia en el indice o las llaves

In [ ]:
for key, value in agent.historico_completo.info_consultas_estructurada.items():
    print(key)
    for n in value: 
        print(n)

: 

In [ ]:
consulta ='''¿Que precio tienen las peliculas en las que ha trabajado el actor o actores con más películas, \
            devuelve el nombre de las películas y el actor o actores que hayan trabajado en ellas cuyos nombres empiezen por las primeras tres letras del abecedario?'''

agent2 = SQLAgent(**pagila)
# print('conexion')
# print(agent.bbdd_name)
# print(agent.historico.contador_interacciones)

tabla_resultado, codigo_sql = agent2.nlp_to_sql(consulta_nlp=consulta, metadata_token_limit= 1000)
agent.close_connection()

In [ ]:
print(agent2.historico_completo.historico)

## Le volvemos a pasar la tabla en formato texto al llm para que nos comente los resultado

In [ ]:
informe = LLMsToDataBase.informe_resultado(consulta_usuario= consulta_pagila_03, tabla_texto=tabla_resultado, max_tokens_respuesta=2000 )
simular_respuesta_generativa(informe)

In [ ]:
llm_to_pagila.close_connection()

In [ ]:
from calcular_tokens import num_tokens_from_string

print(num_tokens_from_string(informe))
print(num_tokens_from_string(tabla_resultado))

In [1]:
import numpy as np 

In [8]:
nombre_historico = 'historico' + str(np.random.randint(low= 0, high= 1_000_000)) +'.txt'
print(nombre_historico)

historico990727.txt
